# Web API for an Interactive Recommender Engine with Apache Spark MLlib using real-world e-commerce data


Vadym Ovcharenko [LinkedIn](https://www.linkedin.com/in/vadymovcharenko/) [GitHub](https://github.com/bolein)


## Abstract
In this project, I used a data sample from a pharmaceutical retail company to prototype two types of popular recommendation models. To make a more realistic use-case, an ETL pipeline was built to enable consistent daily model retraining with new data. Both models were wrapped with REST APIs for access from other applications. This pilot version of a web-based recommendation engine is intended to provide evidence for the company that their data holds unutilized value. It provides meaningful product suggestions and recommendations, although limited with the small size of data. If the customer discovers value in this model, they will consider investing in the development of their Big Data ecosystem and releasing more data for research.

## Introduction
Relevant product recommendations help retailers build a deep relationship with their customers as they give them the sense of being recognized and properly served. Cross-selling of complementary products boosts the revenue stream and increases sales by up to 20%<sup>1</sup>. Surprisingly, the recommendation of news or videos for media, product recommendation or personalization in travel and retail can be handled by similar machine learning algorithms<sup>2</sup>. They are typically classified into two classes  — content-based and collaborative filtering methods. Content-based methods are based on similarity of item attributes and collaborative methods calculate similarity from interactions of similar users. The two approaches produce different types of recommendations<sup>3</sup>. Content-based filtering produces results from a set of similar items, therefore is highly specialized. Collaborative-based filtering is more likely to generate results from different categories, therefore enabling the discovery of niche items from the long tail of the distribution.  Modern recommendation systems combine both methods for achieving effective results.

![image](https://user-images.githubusercontent.com/11277453/49099857-89e33e00-f240-11e8-9551-d45b350c2620.png)


### Collaborative filtering
Collaborative methods work with the interaction matrix that can also be called a rating matrix in the rare case when users provide an explicit rating of items. The task of machine learning is to learn a function that predicts the utility of items to each user. Matrix is typically huge, very sparse where most of the values are missing.

![image](https://user-images.githubusercontent.com/11277453/49100028-f0685c00-f240-11e8-8c49-ad04de5460fa.png)

Traditional collaborative filtering approaches fall into two categories. 

* The user-based approach where recommendations are based on the similarity of user ratings
* The item-based approach where recommendations are based on the item-to-item similarity of ratings.<sup>4</sup>

The most popular algorithms used to solve the utility matrix are k-nearest users/items, matrix factorization, and autoencoders. 

### Content-based filtering
Content-based recommendations rely on characteristics of objects themselves. For this approach, the system has to build a user profile, which is a representation of the user in the item attribute space. Finding the right space of features is not easy and sometimes involves complex NLP techniques or even manual labeling<sup>5</sup>. Content-based filtering has a number of advantages<sup>6</sup> over the collaborative approach. Content-based filtering avoids the cold-start problem that often bedevils collaborative-filtering techniques. New items can be recommended immediately, even if they were not rated by a number of users. 

### Association Rule Mining
Association Rule Mining is another recommendation technique that is different in a way that it treats users. It is less personalized than Collaborative Filtering: it does not account for past interactions, it finds the most frequent purchases among all users and builds rules on top of that<sup>7</sup>. Rules mined should have at least some minimal support and confidence. Support is related to the frequency of occurrence  —  implications of bestsellers have high support. High confidence means that rules are not often violated.

### Other methods
There are many other algorithms like Deep Recommendation, Sequence Prediction, AutoML and Reinforcement Learning in Recommendation<sup>8</sup> as well as hybrid models that combine different algorithms.  

## Big Data Technologies for Recommender Systems
Recommender systems are one of the most successful and widespread applications of machine learning technologies in business. One can find large scale recommender systems in retail, video on demand, or music streaming. In order to develop and maintain such systems, a company typically needs a group of expensive data scientist and engineers. Even though there are some “Recommender as a Service” solutions that might satisfy small and medium e-commerce websites, creating an effective recommender engine requires domain knowledge, deep understanding of the data, and lots of trial-error. Most of the tools used in building such systems are part of the regular toolbox<sup>9</sup> of a Data Scientist. 

* [Azure ML](http://azure.microsoft.com/en-us/services/machine-learning/) machine learning platform to model data and create predictions
* [Amazon Machine Learning](http://aws.amazon.com/machine-learning/) machine learning platform to model data and create predictions
* [Mahout](http://mahout.apache.org/) Hadoop/linear algebra based data mining
* [LightFM](https://github.com/lyst/lightfm) is an actively-developed Python implementation of a number of collaborative- and content-based learning-to-rank recommender algorithms. Using Cython, it easily scales up to very large datasets on multi-core machines and is used in production at a number of companies, including Lyst and Catalant.
* [tensorrec](https://github.com/jfkirk/tensorrec) is a TensorFlow recommendation algorithm and framework in Python

# Implementing a Recommendation System
The goal of this project is to implement a production-ready recommendation engine. Despite the discussed value and widespread of the recommendations in e-commerce, there are very few documented examples<sup>13</sup> of recommender implementations. Most of the articles on the web describe either theoretical or a very basic “Hello World”<sup>10</sup> implementations. Many<sup>8</sup> of the open-source libraries are abandoned, and SaaS solutions do not share the implementation details online. Many of the SaaS recommender systems are implemented on top of Spark. Therefore, for the project, I chose to explore the capabilities of Spark. In particular, it’s Alternating Least Squares (ALS) matrix factorization algorithm implementation and FP-Growth frequent pattern model. In contrast to the web tutorials, I go further than a simple use case and try to create a production-ready implementation. 

## Data collection
### Data extraction
The data is warehoused in the customer’s database and is available using a special bus called E-COM. E-COM is a relatively new technology in the customer's data infrastructure and is actively developed. Because of if this fact, I faced a few problems while working with it. For example, at first, there was no method to dowload the whole collection of transactions. Collaborationg with the E-COM development team, we were able to solve issues quickly.

E-COM exposes a REST API to interact with it. The API is well [documented](https://docs.google.com/document/d/12qB6IpXknP48yfyHkfkvrCxZ-NvEKq-hMchIRLeuHdc/edit#heading=h.91wpu9x8qw). For this project, I used two endpoints specifically:
* [GET /orders](https://docs.google.com/document/d/12qB6IpXknP48yfyHkfkvrCxZ-NvEKq-hMchIRLeuHdc/edit#heading=h.mzszi182k2p7) - Get all orders
* [GET /goods](https://docs.google.com/document/d/12qB6IpXknP48yfyHkfkvrCxZ-NvEKq-hMchIRLeuHdc/edit#heading=h.yclivj7eu413) - Get all goods (with filtering options)

Both models require transaction data. To perform analysis I had to retrieve the data and store it locally. For this purpose I created the following script that downloaded the data from API page by page. I then converted the data to JSON format and stored it on disk. 

E-COM provides sensitive user data, therefore the firwall blocks request for non-whitelisted hosts. To access the E-COM, one IP was whitelisted for our organization, and it belongs to a server that uses E-COM for other purposes. To be able to access E-COM for this project, I had to set up a VPN server on the whitelisted host. I used [this guide](https://www.digitalocean.com/community/tutorials/how-to-set-up-an-openvpn-server-on-ubuntu-16-04#prerequisites) to set up an OpenVPN server on Ubuntu 16.04. This was my first experience with setting up VPN servers and it turned out to be a relatvely simple procedure. I am confused why VPN service providers charge over 10 dollars per month for a [public VPN](https://nordvpn.com) server with limited number of connections, while you can bootstrap a low-cost machine on [Digital Ocean](https://www.digitalocean.com/) and get a dedicated VPN server with UNLIMITED number of connections for less than 10$/month!

In [3]:
page_count = 9999999999999 # init with max value
params = {'page': 1, 'per-page':100}
orders = []

In [4]:
from IPython.display import clear_output, display
import base64
import sys
import time
import requests

def download_all_orders():
    global page_count, orders
    auth_token = 'U2l0ZU96OkFWNzREOA=='
    headers = {'Authorization': 'Basic {}'.format(auth_token)}
    url = 'http://ws.erkapharm.com:8990/ecom/hs/orders?expand=basket'
    
    while params['page'] < page_count:
        started = time.time()
        res = requests.get(url, params=params, headers=headers).json()
        orders += res['orders']
        page_count = res['pageCount']
        params['page'] = params['page'] + 1
        clear_output(wait=True)
        progress = params['page']*1./res['pageCount'] * 100
        ellapsed_seconds = time.time() - started
        left_seconds = ellapsed_seconds * (page_count - params['page'])
        print('progress: {:.2f}% Remaining: {} minutes'.format(progress, int(left_seconds/60)))
        
    return orders

In [8]:
orders = download_all_orders()

In [9]:
len(orders)

99700

In [10]:
import json

with open('orders_complete.json', 'w') as outfile:
    json.dump(orders, outfile)

The downloading process took about 3 hours. To better understand the progress of the task, I calculated the progress rate and ETA. The process took about 3 hours to get about 100,000 records through the API. I fetched 100 records per time. I guess that the process took a long time because of a latency overhead - the VPN server is located in Germany and E-COM instance is in Russia.

The output file is only 63.4 MB, but once again, we're using transaction data only from one of the sources (one website). The company has multiple transaction sources, such as other websites and offline stores (around 1500 stores around the country). But for the purposes of this demo project, we only have access to one of the sources, however, one of the most important ones.

## Data preprocessing

For this project, I chose to use Spark MlLib since it provides a great Python API and uses functional-style operators which are very familiar and very pleasing to me. To test Spark code on my local machine, I used [this guide](https://towardsdatascience.com/how-to-use-pyspark-on-your-computer-9c7180075617) and executed spark code directly in this notebook. From now on, I imagined that I was dealing with a huge amount of data, and used PySpark for all operations

### Transforming phone numbers
As a result of the Data collection step, we have a dataset of well-structured transaction records. However, there are some problems with this data that require additional pre-processing. Since E-COM is a data bus, there data flows from different sources and sometimes does not follow the same conventions. For example, the phone number field, which is thought to be a primary user identifier in transaction data happens in different formats:


In [1]:
## pyspark --conf "spark.mongodb.output.uri=mongodb://127.0.0.1:27017/cs554.associationRules" --packages org.mongodb.spark:mongo-spark-connector_2.11:2.3.1
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
rddjson = sc.textFile('orders_complete.json')
df = sqlContext.read.json(rddjson)

In [2]:
df.registerTempTable('orders')
phoneLengths = spark.sql('SELECT COUNT(*), LENGTH(clientTel), first(clientTel) as example FROM orders GROUP BY LENGTH(clientTel)')
phoneLengths.limit(5).show()

+--------+-----------------+-----------------+
|count(1)|length(clientTel)|          example|
+--------+-----------------+-----------------+
|    4762|               15|  +7(916)539-6205|
|   91516|               17|+7 (905) 545-5709|
|    3422|               11|      79153706507|
+--------+-----------------+-----------------+



We see that there's at least 3 types of different phone formats (depending on the string length). I removed all non-digits.

In [3]:
from pyspark.sql.functions import udf

strToNum = udf(lambda phone: ''.join(filter(str.isdigit, phone)))
df = df.withColumn('clientTelNumeric', strToNum(df.clientTel))
df.select('clientTel', 'clientTelNumeric').limit(10).show()

+-----------------+----------------+
|        clientTel|clientTelNumeric|
+-----------------+----------------+
|+7 (905) 545-5709|     79055455709|
|+7 (915) 182-9688|     79151829688|
|+7 (967) 135-7431|     79671357431|
|+7 (123) 456-7890|     71234567890|
|+7 (123) 456-7890|     71234567890|
|+7 (916) 714-9823|     79167149823|
|+7 (915) 215-0103|     79152150103|
|+7 (926) 172-6883|     79261726883|
|+7 (967) 170-4118|     79671704118|
|+7 (966) 192-6058|     79661926058|
+-----------------+----------------+



### Fitlering out duplicate transactions

There are also some duplicate records in the data. Duplicate transactions can confuse frequent pattern mining algorithms and collaborative filtering models as they will assume that every transaction record is unique. Consequenty, duplicates will artificially increase the frequency of certain datasets. 

In [4]:
df = df.dropDuplicates(['id']).cache()
df.select('clientTelNumeric').distinct().count()/df.count()

0.686037575585981

After parsing only numeric values from the `clientTel` field and dropping duplicate records, we can see that our transaction database has more than **30% of returning customers**. It means that the dataset is a decent case for building collaborative recomendation models.

## Model building


## Building association rules

I decided to start with [Frequent Pattern Mining](https://spark.apache.org/docs/2.3.2/mllib-frequent-pattern-mining.html). I used the FP-growth algorithm, since I studied it in Data Mining class and familiar with how the algorithm works. It turned out, that there's not much information on the Web on how to implement a piece of software using Spark MLlib that will provide association rules as an output of the algorithm. 

To create an FP-Growth model, we need to extract product IDs from transaction records. 

In [42]:
from pyspark.sql.functions import *
# explode items, distinct them and aggregate back (to remove duplicate items in transactions)
transactions = df.select('id', explode('basket.goodsId')) \
                    .groupby('id') \
                    .agg(collect_set('col').alias('items')) \
                    .cache()

In [43]:
from pyspark.ml.fpm import FPGrowth
fp = FPGrowth(minSupport=0.0005, minConfidence=0, numPartitions=4)
fpm = fp.fit(transactions)

Mining association rules takes quite some time. With the `minSupport` property, we can filter out infrequent itemsets. Depending on the implementation of the recommendation engine, it might make sense to mine as many association rules as possible by keeping the `minSupport` as low as possible. If a website has a "frequently bought together" section, it should be filled with items. However, if we want to create a pop-up sreen after click on the "add to cart" with a recommended item, we should have a high confidence in such recommendation.

Confidence is a probability of the consequent given the antecedent (e.g. how likely is the user to purchase consequent in case he already purchased antecedent). Since the dataset is relatively small, we use a low confidence threshold to generate association rules.

In [45]:
rules = fpm.associationRules.sort(desc('confidence'))
rules.show(5)

+----------+----------+-------------------+
|antecedent|consequent|         confidence|
+----------+----------+-------------------+
|   [81976]|   [81977]| 0.5106382978723404|
|  [121286]|  [103180]|0.41626794258373206|
|  [103180]|  [121286]| 0.3096085409252669|
|  [114265]|  [114892]|0.27807486631016043|
|  [121563]|   [97807]| 0.2742616033755274|
+----------+----------+-------------------+
only showing top 5 rows



### Validating results
We generated a table of association rules that can be used by our Web service to give product recommendations. I decided to fetch product names to intuitively evaluate the results of the algorithm.

In [49]:
from pyspark.sql.functions import udf
import requests

def fetchProductNames(ids):
    names = []
    for id in ids:
        url = 'http://138.68.86.83/api/goods/{}?region=78'.format(id)
        res = requests.get(url).json()
        names.append(res['name'])
    return names

In [50]:
pandasRules = rules.limit(5).toPandas()
pandasRules['antecedentName'] = pandasRules.antecedent.map(fetchProductNames)
pandasRules['consequentName'] = pandasRules.consequent.map(fetchProductNames)
pandasRules

,antecedent,consequent,confidence,antecedentName,consequentName
0,[81976],[81977],0.510638,[NotFoundError],[NotFoundError]
1,[121286],[103180],0.416268,[PL Контейнер д/биопроб стер. 60мл с крыш и ло...,[PL Контейнер д/биопроб универс. 120мл полим. ...
2,[103180],[121286],0.309609,[PL Контейнер д/биопроб универс. 120мл полим. ...,[PL Контейнер д/биопроб стер. 60мл с крыш и ло...
3,[114265],[114892],0.278075,[Нью Лайф Бинт марл мед стер 7м х 14см уп N1/СТМ],[Нью Лайф Бинт марл мед стер 5м х 10см уп N1/СТМ]
4,[121563],[97807],0.274262,[NotFoundError],[Ля Рош-Позе Эфаклар Дуо+ Крем-гель корректиру...


These results, indeed, make sense. The first rule is "brilliant green" -> "iodine solution". 

## Collaborative filtering 

### Matrix Factorization
One of the most effective methods of collaborative filtering is Matrix Factorization. Matrix factorization is one of the algorithms from recommender systems family and as the name suggests it factorize a matrix, i.e., decompose a matrix in two(or more) matrices such that once you multiply them you get your original matrix back. In case of the recommendation system, we will typically start with an interaction/rating matrix between users and items and matrix factorization algorithm will decompose this matrix in user and item feature matrix which is also known as embeddings. Example of interaction matrix would be user-movie ratings for movie recommender, user-product purchase flag for transaction data, etc.

![](https://cdn-images-1.medium.com/max/1600/0*Qy8Pku8FJXM60iAs)

Typically user/item embeddings capture latent features about attributes of users and item respectively. Essentially, latent features are the representation of user/item in an arbitrary space which represents how a user rate a movie. In the example of a movie recommender, an example of user embedding might represent affinity of a user to watch serious kind of movie when the value of the latent feature is high and comedy type of movie when the value is low. Similarly, a movie latent feature may have a high value when the movie is more male driven and when it’s more female-driven the value is typically low.

In case of product recomendations, the interaction matrix is very sparse and there are no explicit ratings. The approach used in spark.ml to deal with such data is taken from Collaborative Filtering for Implicit Feedback Datasets<sup>11</sup>. Essentially, instead of trying to model the matrix of ratings directly, this approach treats the data as numbers representing the strength in observations of user actions (such as the number of clicks, or the cumulative duration someone spent viewing a movie). Those numbers are then related to the level of confidence in observed user preferences, rather than explicit ratings given to items. The model then tries to find latent factors that can be used to predict the expected preference of a user for an item.

In this project, I used the total number of purchases for each item as an interaction measure. Alternating Least Squares algorithm implementation in `spark.ml` was used to calculate the latent factors.

In [8]:
# transform the transactions, aggregating total number of purchases per-item, per-user
implicitRatings = df.select('id', 
                            # hash phone numbers to fit in Integer type
                            (col('clientTelNumeric').cast(LongType()) * 67853 % 2**31).alias('telHashed'), 
                            explode('basket').alias('product'), 
                            'product.goodsId', 
                            'product.quantity') \
                    .groupBy('telHashed', 'goodsId') \
                    .agg(sum('quantity').alias('purchases')) \
                    .cache()

In [9]:
implicitRatings.show(5)

+----------+-------+---------+
| telHashed|goodsId|purchases|
+----------+-------+---------+
|1121302175|  16588|        1|
|1569021973| 116978|       30|
|  21874890|  51767|        1|
|  86765306|  26694|        1|
|2022784980| 106619|        2|
+----------+-------+---------+
only showing top 5 rows



In [10]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# split into the train and test set
training, test = implicitRatings.randomSplit([0.8, 0.2])

# train an ALS model
als = ALS(maxIter=5, regParam=0.01, userCol="telHashed", itemCol="goodsId", ratingCol="purchases",
          coldStartStrategy="drop", implicitPrefs=True, nonnegative=True)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="purchases", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
rmse

3.215298356790437

Fitting the best model is challenging and requires a lot of hyperparameter tuning.

One of the most effective use-cases of Matrix Factorization is finding similar items by distance between item embeddings. The item-based approach is proven to be more effective and flexible than other collaborative filtering techniques. It does not require retraining of the whole model for every new interaction and usually yields better results. To implement this, a matrix of distances between item embedding vectors must be computed. Then, top-N similar items could be found for an item. This tasks includes some complex matrix computation and is out of the scope of this project.

### Validation
Validating a recommendation model with implicit recomedations is quite tricky<sup>12</sup>. Basic RMSE does not fit in this case, because there is no information about movies that the user has rated negatively. Therefore, it is suggested to use recall-based metrics for such recommenders. Several metrics have been introduced, the most important being the Mean Percentage Ranking (MPR), also known as Percentile Ranking. However, there is no utility in Spark ML package to evaluate the model in such way. Model validation in tuning is considered outside of the scope of this project.

## Creating a web-service to serve the recommendations

To serve the results over http, I decided to use [Flask](https://github.com/pallets/flask) - a simple web-server framework for python. In my implementation, the app operates in a Spark environment and fetches the data directly from cached Spark SQL dataframes. The app uses a `RecommendationEngine` as the backend for the recommendation computations.

In [ ]:
# engine.py
import os

from pyspark import StorageLevel
from pyspark.ml.recommendation import ALSModel
from pyspark.ml.fpm import FPGrowthModel
from pyspark.sql.functions import *

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class RecommendationEngine:
    """
    A product recommendation engine
    """

    def get_association_rules(self, product_id):
        """
        Searches for association rules
        """
        return self._fpm_model.associationRules\
            .filter(array_contains('antecedent', product_id)) \
            .sort(desc('confidence'))\
            .toJSON()\
            .collect()

    def get_items_for_user(self, phone):
        """
        Searches for item recommendations
        """
        numeric_tel = ''.join(filter(str.isdigit, phone))
        hashed_tel = int(numeric_tel) * 67853 % 2**31
        return self._item_recommendations.filter(col('telHashed') == hashed_tel)\
            .select(explode('recommendations'))\
            .select('col.goodsId', 'col.rating')\
            .toJSON()\
            .collect()

    def __init__(self, sc, model_dir):
        """
        Init the recommendation engine given a Spark context and models path
        """

        logger.info("Starting up the Recommendation Engine: ")

        self.sc = sc

        # Load models
        logger.info("Loading models...")
        als_path = os.path.join(model_dir, 'als_model')
        self._als_model = ALSModel.load(als_path)
        fpm_path = os.path.join(model_dir, 'fpm_model')
        self._fpm_model = FPGrowthModel.load(fpm_path)

        logger.info("Caching precomputed recommendations...")
        self._item_recommendations = self._als_model.recommendForAllUsers(3)\
            .persist(StorageLevel.MEMORY_AND_DISK)

The app module is responsible for routing the requests, querying the recommendation engine and transforming the results to JSON representation.

In [ ]:
# app.py
from flask import Blueprint

main = Blueprint('main', __name__)

import json
from engine import RecommendationEngine

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from flask import Flask, request


@main.route("/products/<int:product_id>/associations", methods=["GET"])
def top_ratings(product_id):
    associations = recommendation_engine.get_association_rules(product_id)
    return (
        '[{}]'.format(','.join(associations)),
        200,
        {'Content-Type': 'application/json; charset=utf-8'}
    )


@main.route("/users/<string:phone_number>/recommendations", methods=["GET"])
def movie_ratings(phone_number):
    recommendations = recommendation_engine.get_items_for_user(phone_number)
    return (
        '[{}]'.format(','.join(recommendations)),
        200,
        {'Content-Type': 'application/json; charset=utf-8'}
    )


def create_app(spark_context, model_dir):
    global recommendation_engine

    recommendation_engine = RecommendationEngine(spark_context, model_dir)

    app = Flask(__name__)
    app.register_blueprint(main)
    return app

As we simulate a production-ready system, we have to ensure the app process is correctly monitored and managed. For this task, I wrapped the app in another web-framework [CherryPy](https://cherrypy.org/). CherryPy monitors the server process and is responsible for thread pooling.

In [ ]:
import time, sys, cherrypy, os
from paste.translogger import TransLogger
from app import create_app
from pyspark import SparkContext, SparkConf


def init_spark_context():
    # load spark context
    conf = SparkConf().setAppName("movie_recommendation-server")
    # IMPORTANT: pass aditional Python modules to each worker
    sc = SparkContext(conf=conf, pyFiles=['engine.py', 'app.py'])

    return sc


def run_server(app):
    # Enable WSGI access logging via Paste
    app_logged = TransLogger(app)

    # Mount the WSGI callable object (app) on the root directory
    cherrypy.tree.graft(app_logged, '/')

    # Set the configuration of the web server
    cherrypy.config.update({
        'engine.autoreload.on': False,
        'log.screen': True,
        'server.socket_port': 5432,
        'server.socket_host': '0.0.0.0'
    })

    # Start the CherryPy WSGI web server
    cherrypy.engine.start()
    cherrypy.engine.block()


if __name__ == "__main__":
    # Init spark context and load libraries
    sc = init_spark_context()
    app = create_app(sc, 'models')

    # start web server
    run_server(app)


    # defining function to run on shutdown
    def stop_spark_context():
        sc.stop()


    cherrypy.engine.subscribe('stop', stop_spark_context)

`spark-submit` is used to start the server. In this project, we use a local Spark session. However, it is easy to scale out by providing a configuration for a real Spark cluster anytime.  

In [41]:
import requests
# request association rules for an item
requests.get('http://localhost:5432/products/97807/associations').json()

[{'antecedent': [97807],
  'consequent': [121563],
  'confidence': 0.20186335403726707}]

In [39]:
# request top-3 product recommendations for a user 
requests.get('http://localhost:5432/users/+7 (967) 170-4118/recommendations').json()

[{'goodsId': 46402, 'rating': 0.020500287},
 {'goodsId': 62448, 'rating': 0.019359678},
 {'goodsId': 116978, 'rating': 0.01613907}]

# Conclusions

In this project, I went through the whole process of a recommender system creation. Starting from data extraction and trasformation, to analysis and model fitting, to production deployment. I used some of the most effective techniques such as Association Rule Mining and ALS Matrix-Factorization Collaborative Filtering.

One of the very important phases that I had to skip because of being limited in time is model validation. The validation is not trivial in case of the implicit ratings, but there are some meaningful metrics that would have been used in a real production pipeline. It is hard to judge the quality of results of this project without such metrics. Considering the low number of association rules mined and huge RMSE for ALS model, I would assume that 100,000 transactions is not sufficient for a recommender system in product recommendations where the product space is tens of thousands of items.

# References and links

### Project links:

* [GitHub Repo](https://github.com/bolein/spark-ml-recommender)
* [Author](https://www.linkedin.com/in/vadymovcharenko/)

### References:

1. "The ROI of Recommendation Engines | CMS Connected." 10 Apr. 2018, https://www.cms-connected.com/News-Archive/April-2018/The-ROI-of-Recommendation-Engines. Accessed 7 Nov. 2018.
1.  "Machine Learning for Recommender systems — Part 1 (algorithms ...." 3 Jun. 2018, https://medium.com/recombee-blog/machine-learning-for-recommender-systems-part-1-algorithms-evaluation-and-cold-start-6f696683d0ed. Accessed 7 Nov. 2018.
1.  "What is the difference between content based filtering and ...." 12 Mar. 2015, https://www.quora.com/What-is-the-difference-between-content-based-filtering-and-collaborative-filtering. Accessed 7 Nov. 2018.
1.  "Item-item collaborative filtering - Wikipedia." https://en.wikipedia.org/wiki/Item-item_collaborative_filtering. Accessed 7 Nov. 2018.
1.  "How Netflix Reverse Engineered Hollywood - The Atlantic." 2 Jan. 2014, https://www.theatlantic.com/technology/archive/2014/01/how-netflix-reverse-engineered-hollywood/282679/. Accessed 7 Nov. 2018.
1.  "Content-based Filtering | Recommender Systems." http://recommender-systems.org/content-based-filtering/. Accessed 7 Nov. 2018.
1.  "How is association rule compared with collaborative filtering in ...." 31 Aug. 2014, https://www.quora.com/How-is-association-rule-compared-with-collaborative-filtering-in-recommender-systems. Accessed 7 Nov. 2018.
1.  "Machine Learning for Recommender systems — Part 2 (Deep ...." 7 Jun. 2018, https://medium.com/recombee-blog/machine-learning-for-recommender-systems-part-2-deep-recommendation-sequence-prediction-automl-f134bc79d66b. Accessed 7 Nov. 2018.
1.  "A List of Recommender Systems and Resources - GitHub." https://github.com/grahamjenson/list_of_recommender_systems. Accessed 7 Nov. 2018.
1.  "Solving business usecases by recommender system using lightFM." 13 Jun. 2018, https://towardsdatascience.com/solving-business-usecases-by-recommender-system-using-lightfm-4ba7b3ac8e62. Accessed 26 Nov. 2018.
1.  "Collaborative Filtering for Implicit Feedback Datasets - Yifan Hu." http://yifanhu.net/PUB/cf.pdf. Accessed 26 Nov. 2018.
1.  "How can I evaluate the implicit feedback ALS algorithm for ...." 1 Oct. 2017, https://stackoverflow.com/questions/46462470/how-can-i-evaluate-the-implicit-feedback-als-algorithm-for-recommendations-in-ap/46490352. Accessed 26 Nov. 2018.
1.  "GitHub - jadianes/spark-movie-lens: An on-line movie recommender ...." https://github.com/jadianes/spark-movie-lens. Accessed 27 Nov. 2018.
